# Prompts & Responses

1. Generate reusable prompt templates
2. Parse responses using pydantic models

In [ ]:
from typing import Literal

from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelName

In [ ]:
# Load API keys from .env file
api_keys = APIKeys()

# Initialize a chat model with your API key
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
)

# 1. Reusable prompt template

Templates offer:
- **Consistency**: Standardize prompts across your application.
- **Maintainability**: Allow you to change the prompt structure in one place instead of throughout your codebase.
- **Modularity**: We can compose multiple message templates together to create reusable templates.
- **Validation**: User input can be validated before injecting into the prompt template
- **Testability**: We can test prompt creation independently

In [ ]:
# Define a simple chat prompt template with input variables
class PoemInputs(BaseModel):
    """Poem prompt input variables.

    Attributes:
        topic (str): The topic of the poem.
        funny_or_sad (Literal["funny", "sad"]): Whether the poem should be funny or sad. Default is 'funny'.
        audience (str): The intended audience for the poem. Default is 'general'.
    """

    topic: str = Field(description="The topic of the poem")
    funny_or_sad: Literal["funny", "sad"] = Field(
        default="funny", description="Whether the poem should be funny or sad. Default is 'funny'."
    )
    audience: str = Field(default="general", description="The intended audience for the poem.")


prompt_template = ChatPromptTemplate.from_template(
    template="Please write me a short poem about {topic}. This poem should be {funny_or_sad} for {audience}.",
)
print("Prompt template created successfully.", "\n", prompt_template)

In [ ]:
# Get message prompt from the template
message_prompt = prompt_template.format_messages(
    topic="getting dressed in the morning", funny_or_sad="funny", audience="children"
)
print("Message prompt formatted successfully.", "\n", message_prompt)

In [ ]:
# Get a different message prompt from the template using PoemVariables model
prompt_variables = PoemInputs(topic="eating soup", funny_or_sad="funny", audience="6 year old boy")
message_prompt = prompt_template.format_messages(**prompt_variables.model_dump(mode="json"))
print("Message prompt formatted successfully.", "\n", message_prompt)

In [ ]:
# Prompt the model and get the response
response = chat_model.invoke(message_prompt)

print("Poem response:", "\n", response.content)

In [ ]:
# Try to create an invalid message prompt from the template
prompt_variables = PoemInputs(topic="eating soup", funny_or_sad="warm and cozy")  # This should raise a validation error
message_prompt = prompt_template.format_messages(**prompt_variables.model_dump(mode="json"))
print("Message prompt formatted successfully.", "\n", message_prompt)

In [ ]:
# Create a template from messages
prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a master poem composer highly skilled a writing poems that elicit the perfect combination of emotion.",
    ),
    ("user", "Please write me a short poem about {topic}. This poem should be {funny_or_sad} for {audience}."),
])
prompt_template

# 2. Enforce structured output

In [ ]:
# Define the response model for structured output
class PoemOutputs(BaseModel):
    """Poem response output variables.

    Attributes:
        title (str): The title of the poem.
        content (str): The content of the poem.
        voice (str | None): Recommended voice for reading the poem aloud. Optional.
    """

    title: str = Field(description="The title of the poem")
    content: str = Field(description="The content of the poem.")
    voice: str | None = Field(default=None, description="Recommended voice for reading the poem aloud. Optional.")

## Option A: `PydanticOutputParser` & format instructions

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

# Create a Pydantic output parser for the response model
response_parser = PydanticOutputParser(pydantic_object=PoemOutputs)

# Create a prompt template with response format instructions
prompt_template_w_format = ChatPromptTemplate.from_template(
    template=(
        "Please write me a short poem about {topic}. This poem should be {funny_or_sad} for {audience}."
        "\n\n{format_instructions}"
    ),
    partial_variables={"format_instructions": response_parser.get_format_instructions()},
)

In [ ]:
# Create a chain combining the prompt, chat model, and response parser
chain = prompt_template_w_format | chat_model | response_parser

In [ ]:
# Invoke the chain with prompt variables
prompt_variables = PoemInputs(topic="eating soup", funny_or_sad="funny", audience="6 year old boy")
response: PoemOutputs = chain.invoke(prompt_variables.model_dump(mode="json"))

In [ ]:
# Display the structured response
print("Poem Title:", response.title)
print("\nPoem Content:", response.content)
print("\nRecommended Voice:", response.voice)

## Option B: New model `with_structured_output`

 I'd strongly recommend using `.with_structured_output()` instead of `PydanticOutputParser` for Claude models. It's more reliable because it uses Claude's native tool-calling rather than trying to parse JSON from text.

In [ ]:
# Wrap the chat model to enforce structured output
structured_model = chat_model.with_structured_output(PoemOutputs)

# Create a chain combining the (original) prompt template and the structured model
chain = prompt_template | structured_model

In [ ]:
# Invoke the chain with prompt variables
response: PoemOutputs = chain.invoke(PoemInputs(topic="playing Lego with my sister").model_dump(mode="json"))

In [ ]:
# Display the structured response
print("Poem Title:", response.title)
print("\nPoem Content:", response.content)
print("\nRecommended Voice:", response.voice)

# Example: Review parser

Build a parsers for analyzing book reviews to determine if the review is:

1. About the book in question
2. Overall {Negative, Positive, or Neural}
3. Pros of the book
4. Cons of the book
5. Other books to consider


Here we will use a response model with rich field descriptions help the model understand exactly what we want.
- Field descriptions can be used to provide detailed instructions to the LLM with structure
  
  ```python
  quality_review: bool = Field(
        description=(
            "Whether the review is a quality review that provides useful information about the book's content. "
            "Instead of just focusing on extraneous details, like shipping speed or paper quality."
        )
    )
  ```
- You can add examples in descriptions if needed for clarity
- Use the response model's class docstring for overall context


In [ ]:
class BookReviewInputs(BaseModel):
    """Book review prompt input variables.

    Attributes:
        book_title (str): The title of the book being reviewed.
        book_overview (str): A brief overview of the book.
        review_title (str): The title of the book review.
        review_text (str): The text of the book review.
    """

    book_title: str = Field(description="The title of the book being reviewed.")
    book_overview: str = Field(description="A brief overview of the book from the publisher.")
    review_title: str = Field(description="The title of the book review.")
    review_text: str = Field(description="The text of the book review.")


class BookReviewOutputs(BaseModel):
    """Book review analysis output variables.

    Attributes:
        is_about_book (bool): Whether the review is about the book in question.
        quality_review (bool): Whether the review is a quality review.
        overall_sentiment (Literal["Negative", "Positive", "Neutral"]): The overall sentiment of the review.
        pros (str): The pros of the book mentioned in the review.
        cons (str): The cons of the book mentioned in the review.
        other_books_to_consider (str): Other books to consider mentioned in the review.
    """

    is_about_book: bool = Field(description="Whether the review is about the book in question.")
    quality_review: bool = Field(
        description=(
            "Whether the review is a quality review that provides useful information about the book's content. "
            "Instead of just focusing on extraneous details, like shipping speed or paper quality."
        )
    )
    overall_sentiment: Literal["Negative", "Positive", "Neutral"] = Field(
        description="The overall sentiment of the review."
    )
    pros: list[str] = Field(description="The pros of the book mentioned in the review.")
    cons: list[str] = Field(description="The cons of the book mentioned in the review.")
    other_books_to_consider: list[str] = Field(description="Other books to consider mentioned in the review.")


book_review_template = ChatPromptTemplate.from_template(
    template=(
        "You are an expert book review analyst. "
        "Given the book details, analyze the review relative to the book overview. "
        "Extract the requested information in a structured format.\n\n"
        "Analyze this book review:\n"
        "Book Title: {book_title}\n"
        "Book Overview: {book_overview}\n"
        "Review Title: {review_title}\n"
        "Review Text: {review_text}\n\n"
    ),
)

# Wrap the chat model to enforce structured output
chat_model.max_tokens = 4096
book_reviewer_model = chat_model.with_structured_output(BookReviewOutputs)

# Create a chain combining the prompt template and the book reviewer model
book_review_chain = book_review_template | book_reviewer_model

In [ ]:
book_title = (
    "Generative AI with LangChain: "
    "Build production-ready LLM applications and advanced agents using Python, LangChain, and LangGraph"
)
book_overview = """
This second edition tackles the biggest challenge facing companies in AI today: moving from prototypes to production.
Fully updated to reflect the latest developments in the LangChain ecosystem, it captures how modern AI systems are
developed, deployed, and scaled in enterprise environments. This edition places a strong focus on multi-agent
architectures, robust LangGraph workflows, and advanced retrieval-augmented generation (RAG) pipelines.

You'll explore design patterns for building agentic systems, with practical implementations of multi-agent setups
for complex tasks. The book guides you through reasoning techniques such as Tree-of -Thoughts, structured generation,
and agent handoffs—complete with error handling examples. Expanded chapters on testing, evaluation, and deployment
address the demands of modern LLM applications, showing you how to design secure, compliant AI systems with
built-in safeguards and responsible development principles. This edition also expands RAG coverage with guidance on
hybrid search, re-ranking, and fact-checking pipelines to enhance output accuracy.

What you will learn
Design and implement multi-agent systems using LangGraph
Implement testing strategies that identify issues before deployment
Deploy observability and monitoring solutions for production environments
Build agentic RAG systems with re-ranking capabilities
Architect scalable, production-ready AI agents using LangGraph and MCP
Work with the latest LLMs and providers like Google Gemini, Anthropic, Mistral, DeepSeek, and OpenAI's o3-mini
Design secure, compliant AI systems aligned with modern ethical practices
Who this book is for
This book is for developers, researchers, and anyone looking to learn more about LangChain and LangGraph.
With a strong emphasis on enterprise deployment patterns, it's especially valuable for teams implementing LLM solutions
at scale. While the first edition focused on individual developers, this updated edition expands its reach to support
engineering teams and decision-makers working on enterprise-scale LLM strategies. A basic understanding of Python is
required, and familiarity with machine learning will help you get the most out of this book.

Table of Contents
The Rise of Generative AI: From Language Models to Agents
First Steps with LangChain
Building Workflows with LangGraph
Building Intelligent RAG Systems with LangChain
Building Intelligent Agents
Advanced Applications and Multi-Agent Systems
Software Development and Data Analysis Agents
Evaluation and Testing
Observability and Production Deployment
The Future of LLM Applications
"""

# Define some example reviews
reviews = [
    (
        "Modern LLM Practical Guide",
        "Generative AI with LangChain is a comprehensive, technically rich, and highly practical guide for developers "
        "and architects building LLM applications. it elaborates LangGraph—a powerful orchestration framework built on "
        "top of LangChain—and teaches how to use it to build structured, multi-agent workflows capable of reasoning, "
        "coordination, and error recovery. One of its greatest strengths lies in demystifying the journey from "
        "prototype to production by introducing scalable design patterns that incorporate advanced techniques like "
        "retrieval-augmented generation (RAG), hybrid search, re-ranking, and fact-checking. Readers are guided "
        "through intelligent agent construction using LangGraph's node and edge abstractions, bringing determinism "
        "and modularity to systems that are otherwise probabilistic and fragile. A major focus of the book is showing "
        "how to architect robust workflows that go beyond one-shot queries or simple chatbots. It includes examples "
        "of specialized agents for tasks such as data analysis and software development—use cases that are "
        "increasingly being adopted across industries. Moreover, the book doesn't shy away from difficult topics "
        "like observability, monitoring, and testing in dynamic LLM environments. Concepts such as latency tracing, "
        "model output evaluation, and prompt governance are treated as essential, not optional. For teams concerned "
        "with responsible AI practices, there's also guidance on embedding security, compliance, and ethical "
        "development into the system design from the outset. While it assumes basic Python skills, it remains "
        "friendly to intermediate readers, and the advanced topics will be particularly valuable for experienced "
        "teams working on critical production systems. Overall, this second edition serves as a field guide for "
        "anyone serious about building intelligent, scalable, and trustworthy generative AI applications. It is an "
        "essential resource for moving from lab experiments to reliable deployments, offering practical blueprints "
        "for architecting AI agents that can reason, interact, and perform in real-world enterprise environments. "
        "Whether you're working solo or as part of a cross-functional platform team, The inclusion of multi-agent "
        "coordination patterns—where multiple agents can specialize, delegate, and hand off tasks—offers a glimpse "
        "into what the future of enterprise AI may look like. this book will give you the patterns, tools, and "
        "confidence to deliver LLM-powered solutions that are not only smart, but stable and secure.",
    ),
    (
        "Great Resource for those working for Lang Chain",
        "I'm working on building some AI products for work. This has been a great resource to learn how to properly "
        "leverage LangChain and LangGraph when building AI products using GenAI.",
    ),
    ("Felt a bit deceived!", "Says “In color” but it has no color!"),
]

# Create inputs for the book review chain
inputs = [
    BookReviewInputs(
        book_title=book_title, book_overview=book_overview, review_title=review_title, review_text=review_text
    )
    for review_title, review_text in reviews
]

In [ ]:
# Analyze the book reviews
analysis: list[BookReviewOutputs] = book_review_chain.batch([ii.model_dump(mode="json") for ii in inputs])

In [ ]:
# Print the analysis results
for i, result in enumerate(analysis):
    print(f"\nAnalysis for Review {i + 1}:")
    print(result.model_dump_json(indent=2))